In [2]:
import os
from google.colab import drive

import pandas as pd
import numpy as np


from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier


from xgboost import plot_importance
import sklearn
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline



In [3]:
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/1.누수감지데이터-통합(leaks-all).csv")


df.isna().sum()
df = df.dropna()
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,site,sid,ldate,lrate,llevel,leaktype,0HZ,10HZ,20HZ,30HZ,...,MAX10,MAX11,MAX12,MAX13,MAX14,MAX15,MAX16,MAX17,MAX18,MAX19
0,S-2711010100,S-0359369084021423,20200903,90,695,out,0,3,0,3,...,280,741,280,469,270,813,280,644,270,250
1,S-2711010100,S-0359369084021423,20200907,90,261,out,0,5,4,4,...,280,629,280,212,270,234,270,355,270,223
2,S-2711010100,S-0359369084044425,20200704,90,747,out,0,6,6,6,...,410,985,410,596,410,793,410,785,410,846
3,S-2711010100,S-0359369084044425,20200705,90,747,out,0,6,6,6,...,410,758,410,857,420,650,410,907,410,631
4,S-2711010100,S-0359369084044425,20200706,90,825,out,0,6,6,6,...,410,771,410,622,410,894,420,775,410,918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78199,S-4571025029,S-0359369084052139,20200615,40,148,normal,0,4,4,4,...,270,101,250,461,250,115,260,119,250,167
78200,S-4571025029,S-0359369084052139,20200616,50,146,normal,0,4,4,4,...,250,125,250,102,250,106,250,156,250,141
78201,S-4571025029,S-0359369084052139,20200617,70,326,normal,0,7,7,6,...,250,384,230,862,250,256,260,443,250,283
78202,S-4571025029,S-0359369084052139,20200619,60,388,normal,0,8,8,10,...,260,196,250,85,270,106,270,106,280,56


In [4]:
data1 = df.iloc[:,6:]
data1

#"Before splitting, organize and concatenate the remaining data
#except for the data corresponding to the y-values(for training.)"

,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,90HZ,...,MAX10,MAX11,MAX12,MAX13,MAX14,MAX15,MAX16,MAX17,MAX18,MAX19
0,0,3,0,3,3,3,3,5,5,5,...,280,741,280,469,270,813,280,644,270,250
1,0,5,4,4,3,4,4,6,4,6,...,280,629,280,212,270,234,270,355,270,223
2,0,6,6,6,6,6,6,6,6,9,...,410,985,410,596,410,793,410,785,410,846
3,0,6,6,6,6,6,6,8,8,8,...,410,758,410,857,420,650,410,907,410,631
4,0,6,6,6,6,6,6,6,6,9,...,410,771,410,622,410,894,420,775,410,918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78199,0,4,4,4,4,4,4,3,4,4,...,270,101,250,461,250,115,260,119,250,167
78200,0,4,4,4,4,4,4,4,4,4,...,250,125,250,102,250,106,250,156,250,141
78201,0,7,7,6,7,6,6,6,7,7,...,250,384,230,862,250,256,260,443,250,283
78202,0,8,8,10,8,9,5,10,9,8,...,260,196,250,85,270,106,270,106,280,56


In [5]:
data2 = df.iloc[:,0:5]
data2

,site,sid,ldate,lrate,llevel
0,S-2711010100,S-0359369084021423,20200903,90,695
1,S-2711010100,S-0359369084021423,20200907,90,261
2,S-2711010100,S-0359369084044425,20200704,90,747
3,S-2711010100,S-0359369084044425,20200705,90,747
4,S-2711010100,S-0359369084044425,20200706,90,825
...,...,...,...,...,...
78199,S-4571025029,S-0359369084052139,20200615,40,148
78200,S-4571025029,S-0359369084052139,20200616,50,146
78201,S-4571025029,S-0359369084052139,20200617,70,326
78202,S-4571025029,S-0359369084052139,20200619,60,388


In [6]:
data = pd.concat([data2,data1],axis=1)
data

,site,sid,ldate,lrate,llevel,0HZ,10HZ,20HZ,30HZ,40HZ,...,MAX10,MAX11,MAX12,MAX13,MAX14,MAX15,MAX16,MAX17,MAX18,MAX19
0,S-2711010100,S-0359369084021423,20200903,90,695,0,3,0,3,3,...,280,741,280,469,270,813,280,644,270,250
1,S-2711010100,S-0359369084021423,20200907,90,261,0,5,4,4,3,...,280,629,280,212,270,234,270,355,270,223
2,S-2711010100,S-0359369084044425,20200704,90,747,0,6,6,6,6,...,410,985,410,596,410,793,410,785,410,846
3,S-2711010100,S-0359369084044425,20200705,90,747,0,6,6,6,6,...,410,758,410,857,420,650,410,907,410,631
4,S-2711010100,S-0359369084044425,20200706,90,825,0,6,6,6,6,...,410,771,410,622,410,894,420,775,410,918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78199,S-4571025029,S-0359369084052139,20200615,40,148,0,4,4,4,4,...,270,101,250,461,250,115,260,119,250,167
78200,S-4571025029,S-0359369084052139,20200616,50,146,0,4,4,4,4,...,250,125,250,102,250,106,250,156,250,141
78201,S-4571025029,S-0359369084052139,20200617,70,326,0,7,7,6,7,...,250,384,230,862,250,256,260,443,250,283
78202,S-4571025029,S-0359369084052139,20200619,60,388,0,8,8,10,8,...,260,196,250,85,270,106,270,106,280,56


In [7]:
y_data = df['leaktype']
y_data

0           out
1           out
2           out
3           out
4           out
          ...  
78199    normal
78200    normal
78201    normal
78202    normal
78203    normal
Name: leaktype, Length: 78204, dtype: object

In [8]:
y_data.unique()

array(['out', 'in', 'noise', 'other', 'normal'], dtype=object)

In [9]:
y_data = y_data.replace('out',0)
y_data = y_data.replace('in',1)
y_data = y_data.replace('noise',2)
y_data = y_data.replace('other',3)
y_data = y_data.replace('normal',4)

y_data

0        0
1        0
2        0
3        0
4        0
        ..
78199    4
78200    4
78201    4
78202    4
78203    4
Name: leaktype, Length: 78204, dtype: int64

In [10]:
y_data.value_counts()

4    24628
0    21923
1    16591
3     8775
2     6287
Name: leaktype, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
target = y_data

x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size = 0.2,
                                                      shuffle=True, stratify = target, random_state = 34)
x_train                                             

,site,sid,ldate,lrate,llevel,0HZ,10HZ,20HZ,30HZ,40HZ,...,MAX10,MAX11,MAX12,MAX13,MAX14,MAX15,MAX16,MAX17,MAX18,MAX19
64344,S-4413110100,S-0359369084110267,20200530,60,141,0,4,5,4,4,...,360,127,360,125,360,112,360,112,360,158
32586,S-4677025033,S-0359369083957122,20190623,90,444,0,9,9,11,9,...,300,742,300,461,300,460,300,481,300,410
35863,S-4784025026,S-0359369085191258,20200901,90,329,0,14,12,15,14,...,4770,220,260,380,260,360,260,379,260,328
45308,S-4372025021,S-0359369085145411,20201227,90,326,0,2,4,4,4,...,260,63,250,570,250,152,250,409,250,208
57307,S-2711010100,S-0359369085090369,20200915,30,137,0,5,5,4,5,...,290,28,300,125,300,150,300,110,290,138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2856,S-4571025029,S-0359369084037536,20200814,90,868,0,0,0,0,3,...,250,678,250,916,250,971,250,907,250,718
8851,S-4677025033,S-0359369081307676,20190609,90,580,0,2,2,2,2,...,450,741,450,527,450,647,450,493,450,477
24560,S-4413110100,S-0359369084099742,20200429,90,198,0,16,16,16,16,...,420,288,2770,215,2770,214,2790,162,2800,159
76402,S-4571025029,S-0359369084049788,20200809,70,343,0,4,3,4,4,...,270,410,270,333,270,146,270,246,270,59


In [12]:
mmscaler = MinMaxScaler()
train_data_scale = mmscaler.fit_transform(x_train.iloc[:,3:])
train_data_scale = pd.DataFrame(train_data_scale, columns=x_train.iloc[:,3:].columns)

In [13]:
train_data_scale

,lrate,llevel,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,...,MAX10,MAX11,MAX12,MAX13,MAX14,MAX15,MAX16,MAX17,MAX18,MAX19
0,0.500000,0.022279,0.0,0.046512,0.034247,0.032,0.018349,0.015528,0.003276,0.029070,...,0.066532,0.071472,0.066398,0.067678,0.066398,0.060983,0.066398,0.059884,0.066398,0.087596
1,1.000000,0.215150,0.0,0.104651,0.061644,0.088,0.041284,0.034161,0.009009,0.075581,...,0.054435,0.443973,0.054326,0.263711,0.054326,0.267022,0.054326,0.274419,0.054326,0.235744
2,1.000000,0.141948,0.0,0.162791,0.082192,0.120,0.064220,0.043478,0.011466,0.087209,...,0.955645,0.127801,0.046278,0.216453,0.046278,0.207815,0.046278,0.215116,0.046278,0.187537
3,1.000000,0.140038,0.0,0.023256,0.027397,0.032,0.018349,0.015528,0.003276,0.023256,...,0.046371,0.032707,0.044266,0.327305,0.044266,0.084665,0.044266,0.232558,0.044266,0.116990
4,0.000000,0.019733,0.0,0.058140,0.034247,0.032,0.022936,0.015528,0.004095,0.029070,...,0.052419,0.011508,0.054326,0.067678,0.054326,0.083481,0.054326,0.058721,0.052314,0.075838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62558,1.000000,0.485041,0.0,0.000000,0.000000,0.000,0.013761,0.000000,0.000000,0.017442,...,0.044355,0.405209,0.044266,0.529172,0.044266,0.569568,0.044266,0.522093,0.044266,0.416814
62559,1.000000,0.301719,0.0,0.023256,0.013699,0.016,0.009174,0.006211,0.001638,0.011628,...,0.084677,0.443368,0.084507,0.302217,0.084507,0.377738,0.084507,0.281395,0.084507,0.275132
62560,1.000000,0.058561,0.0,0.186047,0.109589,0.128,0.073394,0.046584,0.013923,0.110465,...,0.078629,0.168988,0.551308,0.120187,0.551308,0.121374,0.555332,0.088953,0.557344,0.088183
62561,0.666667,0.150859,0.0,0.046512,0.020548,0.032,0.018349,0.012422,0.003276,0.017442,...,0.048387,0.242883,0.048290,0.189032,0.048290,0.081113,0.048290,0.137791,0.048290,0.029394


In [14]:
test_data_scale = mmscaler.transform(x_valid.iloc[:,3:])
test_data_scale = pd.DataFrame(test_data_scale, columns=x_valid.iloc[:,3:].columns)
test_data_scale

In [15]:
train_data_scale = train_data_scale.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
traindata = pd.concat([train_data_scale,y_train], axis=1)
traindata

,lrate,llevel,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,...,MAX11,MAX12,MAX13,MAX14,MAX15,MAX16,MAX17,MAX18,MAX19,leaktype
0,0.500000,0.022279,0.0,0.046512,0.034247,0.032,0.018349,0.015528,0.003276,0.029070,...,0.071472,0.066398,0.067678,0.066398,0.060983,0.066398,0.059884,0.066398,0.087596,4
1,1.000000,0.215150,0.0,0.104651,0.061644,0.088,0.041284,0.034161,0.009009,0.075581,...,0.443973,0.054326,0.263711,0.054326,0.267022,0.054326,0.274419,0.054326,0.235744,1
2,1.000000,0.141948,0.0,0.162791,0.082192,0.120,0.064220,0.043478,0.011466,0.087209,...,0.127801,0.046278,0.216453,0.046278,0.207815,0.046278,0.215116,0.046278,0.187537,1
3,1.000000,0.140038,0.0,0.023256,0.027397,0.032,0.018349,0.015528,0.003276,0.023256,...,0.032707,0.044266,0.327305,0.044266,0.084665,0.044266,0.232558,0.044266,0.116990,3
4,0.000000,0.019733,0.0,0.058140,0.034247,0.032,0.022936,0.015528,0.004095,0.029070,...,0.011508,0.054326,0.067678,0.054326,0.083481,0.054326,0.058721,0.052314,0.075838,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62558,1.000000,0.485041,0.0,0.000000,0.000000,0.000,0.013761,0.000000,0.000000,0.017442,...,0.405209,0.044266,0.529172,0.044266,0.569568,0.044266,0.522093,0.044266,0.416814,0
62559,1.000000,0.301719,0.0,0.023256,0.013699,0.016,0.009174,0.006211,0.001638,0.011628,...,0.443368,0.084507,0.302217,0.084507,0.377738,0.084507,0.281395,0.084507,0.275132,0
62560,1.000000,0.058561,0.0,0.186047,0.109589,0.128,0.073394,0.046584,0.013923,0.110465,...,0.168988,0.551308,0.120187,0.551308,0.121374,0.555332,0.088953,0.557344,0.088183,1
62561,0.666667,0.150859,0.0,0.046512,0.020548,0.032,0.018349,0.012422,0.003276,0.017442,...,0.242883,0.048290,0.189032,0.048290,0.081113,0.048290,0.137791,0.048290,0.029394,4


In [16]:
test_data_scale = test_data_scale.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)
testdata = pd.concat([test_data_scale,y_valid], axis=1)
testdata

,lrate,llevel,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,...,MAX11,MAX12,MAX13,MAX14,MAX15,MAX16,MAX17,MAX18,MAX19,leaktype
0,0.500000,0.033736,0.0,0.046512,0.027397,0.040,0.018349,0.012422,0.003276,0.023256,...,0.057541,0.048290,0.090432,0.048290,0.063943,0.048290,0.090698,0.048290,0.098178,4
1,1.000000,0.066836,0.0,0.267442,0.143836,0.200,0.100917,0.071429,0.022113,0.186047,...,0.116293,0.074447,0.109102,0.853119,0.134991,0.074447,0.119186,0.074447,0.103469,3
2,1.000000,0.224698,0.0,0.023256,0.020548,0.024,0.009174,0.006211,0.002457,0.011628,...,0.252574,0.082495,0.312719,0.078471,0.236234,0.078471,0.267442,0.078471,0.219283,3
3,1.000000,0.160407,0.0,0.034884,0.027397,0.032,0.013761,0.012422,0.002457,0.023256,...,0.258631,0.040241,0.057760,0.040241,0.131439,0.040241,0.148837,0.040241,0.111699,0
4,0.000000,0.128581,0.0,0.046512,0.027397,0.032,0.018349,0.012422,0.003276,0.023256,...,0.058147,0.046278,0.123104,0.046278,0.015394,0.046278,0.029070,0.046278,0.028219,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15636,0.666667,0.061744,0.0,0.046512,0.034247,0.040,0.018349,0.012422,0.004914,0.023256,...,0.112659,0.074447,0.180280,0.076459,0.071048,0.076459,0.114535,0.072435,0.032922,4
15637,1.000000,0.150859,0.0,0.058140,0.041096,0.032,0.018349,0.015528,0.004095,0.034884,...,0.244094,0.052314,0.137106,0.052314,0.381291,0.052314,0.144186,0.052314,0.142857,0
15638,0.166667,0.068746,0.0,0.046512,0.027397,0.032,0.018349,0.015528,0.004095,0.023256,...,0.020594,0.044266,0.010502,0.044266,0.111901,0.044266,0.013372,0.044266,0.011170,4
15639,1.000000,0.331636,0.0,0.069767,0.041096,0.048,0.027523,0.012422,0.004914,0.034884,...,0.354331,0.066398,0.280630,0.068410,0.348135,0.068410,0.338953,0.068410,0.459730,1


In [ ]:
lgb = LGBMClassifier(random_state=42)
lgbm_params = {
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'num_leaves': [31, 63, 127]
}

lgb_random = RandomizedSearchCV(lgb, param_distributions=lgbm_params, n_iter=10, scoring='accuracy', 
                                verbose=3, n_jobs=-1, cv=3, random_state=42)
lgb_random.fit(train_data_scale, y_train)



Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=LGBMClassifier(random_state=42), n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.1, 0.15],
                                        'max_depth': [3, 5, 7],
                                        'n_estimators': [50, 100, 200],
                                        'num_leaves': [31, 63, 127]},
                   random_state=42, scoring='accuracy', verbose=3)

In [ ]:
lgb_pred = lgb_random.predict(test_data_scale)
print("Test Accuracy: %f" % accuracy_score(y_valid, lgb_pred))

print(lgb_random.best_params_)


Test Accuracy: 0.910428
{'num_leaves': 63, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.15}


In [ ]:
for i in range(20):
  print("True:", y_valid[i],',Predict:',lgb_pred[i])

True: 4 ,Predict: 4
True: 3 ,Predict: 3
True: 3 ,Predict: 1
True: 0 ,Predict: 0
True: 4 ,Predict: 4
True: 4 ,Predict: 4
True: 1 ,Predict: 1
True: 4 ,Predict: 4
True: 4 ,Predict: 4
True: 4 ,Predict: 4
True: 4 ,Predict: 4
True: 1 ,Predict: 1
True: 0 ,Predict: 0
True: 1 ,Predict: 3
True: 1 ,Predict: 1
True: 0 ,Predict: 0
True: 0 ,Predict: 0
True: 0 ,Predict: 1
True: 0 ,Predict: 0
True: 4 ,Predict: 4
